# Figure 2 & Figure 2 Supplementary Material & Figure 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import square
from scipy.stats import ks_2samp
from matplotlib.backends.backend_pdf import PdfPages

## Define three node network

In [ ]:
def three_node_network(f):
    # Define model parameters
    T = 100000                  # Total number of time steps.
    dt= 0.001                   # Sampling interval (e.g., 1 ms)
    t = np.arange(0,T)*dt       # Time axis
    
    K = 100                     # For K repeats, compute the target readout for:
    readout_AT = np.zeros(K)    #... Input A   to Target,
    readout_BT = np.zeros(K)    #... Input B   to Target,
    readout_ABT= np.zeros(K)    #... Input A&B to Target.
    for k in np.arange(K):
        phi0    = 2*np.pi*np.random.rand()
        input_A = (square(2*np.pi*f["f_IA"]*t + 0.2*np.random.randn() + phi0, 0.1)+1)/2
        input_B = (square(2*np.pi*f["f_IB"]*t + 0.2*np.random.randn() + phi0, 0.1)+1)/2
        target  =  np.sin(2*np.pi*f["f_T"] *t + 0.2*np.random.randn() + phi0)
        
        readout_AT[k] = np.abs(np.sum(input_A * target))          / T
        readout_BT[k] = np.abs(np.sum(input_B * target))          / T
        readout_ABT[k]= np.abs(np.sum(input_A * input_B * target))/ T
    
    # Save the readout results.
    readout = dict.fromkeys(["AT","BT","ABT"])
    readout["AT"] =readout_AT
    readout["BT"] =readout_BT
    readout["ABT"]=readout_ABT
    # ... and save an example trace.
    ex     = dict.fromkeys(["A","B","T"])
    ex["A"]       =input_A
    ex["B"]       =input_B
    ex["T"]       =target
    ex["t"]       =t
    
    return readout,ex

# Figure 2

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=80)

# (1,2,1) and (1,2,2)
f = {"f_IA":1, "f_IB":2, "f_T" :1};
readout_ABA,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":2, "f_T" :2};
readout_ABB,ex=three_node_network(f)
plt.subplot(323); plt.boxplot([readout_ABA["BT"],readout_ABB["BT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$2 \rightarrow 1$", r"$2 \rightarrow 2$"]); plt.ylabel("Readout at target")
plt.subplot(325); plt.boxplot([readout_ABA["AT"],readout_ABB["AT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$1 \rightarrow 1$", r"$1 \rightarrow 2$"]); plt.ylabel("Readout at target")

# (1,phi,1) and (1,phi,phi)
golden = (1+5**0.5)/2
f = {"f_IA":1, "f_IB":golden, "f_T":1};
readout_ABA,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":golden, "f_T":golden};
readout_ABB,ex=three_node_network(f)
plt.subplot(324); plt.boxplot([readout_ABA["BT"],readout_ABB["BT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$\phi \rightarrow 1$", r"$\phi \rightarrow \phi$"])
plt.subplot(326); plt.boxplot([readout_ABA["AT"],readout_ABB["AT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$1 \rightarrow 1$",     r"$1 \rightarrow \phi$"]);

# include example trace
plt.subplot(311);
plt.plot(ex["t"],ex["A"]+2.2,'r',         label="Input A");  # Input A, f=1
plt.plot(ex["t"],ex["B"]+1.1,'xkcd:gold', label="Input B");  # Input B, f=Golden ratio
plt.plot(ex["t"],ex["T"],    'k',         label="Target");   # Target,  f=Golden ratio
plt.xlim([20,25]); plt.axis("off"); plt.legend();
fig.savefig("Figure-2.pdf", bbox_inches='tight')


## Figure 2 Supplementary Material

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=80)

ratio = np.exp(1)
f = {"f_IA":1, "f_IB":ratio, "f_T":1};      readout_ABA,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":ratio, "f_T":ratio};  readout_ABB,ex=three_node_network(f)
plt.subplot(222); plt.boxplot([readout_ABA["BT"],readout_ABB["BT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$e \rightarrow 1$", r"$e \rightarrow e$"]); plt.ylabel("Readout at target")
plt.subplot(224); plt.boxplot([readout_ABA["AT"],readout_ABB["AT"]]); plt.ylim([-0.01,0.1]); plt.xticks([1,2],[r"$1 \rightarrow 1$", r"$1 \rightarrow e$"]); plt.ylabel("Readout at target");
fig.savefig("Figure-2-Supp-Material.pdf", bbox_inches='tight')


# Figure 3

In [ ]:
fig = plt.figure(figsize=(10, 3), dpi=80)

# Golden ratio (1,phi,phi^2)
ratio = (1+5**0.5)/2
f = {"f_IA":1, "f_IB":ratio, "f_T":1};        readout_1,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":ratio, "f_T":ratio};    readout_2,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":ratio, "f_T":ratio**2}; readout_3,ex=three_node_network(f)

plt.subplot(121); plt.boxplot([readout_1["ABT"],readout_2["ABT"],readout_3["ABT"]]); plt.ylim([-0.001,0.02]);
plt.xticks([1,2,3],[r"$(1,\phi) \rightarrow 1$", r"$(1,\phi) \rightarrow \phi$", r"$(1,\phi) \rightarrow \phi^2$"]); plt.ylabel("Readout at target")

ks13_golden = ks_2samp(readout_1["ABT"], readout_3["ABT"])
ks23_golden = ks_2samp(readout_2["ABT"], readout_3["ABT"])
print("Golden: Target=1 vs Target=\phi^2", ks13_golden)
print("Golden: Target=\phi vs Target=\phi^2", ks23_golden)

# Integer ratio (1,2,3)
f = {"f_IA":1, "f_IB":2, "f_T":1};  readout_1,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":2, "f_T":2};  readout_2,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":2, "f_T":3};  readout_3,ex=three_node_network(f)
plt.subplot(122); plt.boxplot([readout_1["ABT"],readout_2["ABT"],readout_3["ABT"]])
plt.ylim([-0.001,0.02]); plt.xticks([1,2,3],[r"$(1,2) \rightarrow 1$", r"$(1,2) \rightarrow 2$", r"$(1,2) \rightarrow 3$"])

ks13_integer = ks_2samp(readout_1["ABT"], readout_3["ABT"])
ks23_integer = ks_2samp(readout_2["ABT"], readout_3["ABT"])
print("Integer: Target=1 vs Target=3", ks13_integer)
print("Integer: Target=2 vs Target=3", ks23_integer)

fig.savefig("Figure-3.pdf", bbox_inches='tight')


## Figure 3 Supplementary Material

In [ ]:
fig = plt.figure(figsize=(10, 3), dpi=80)

# Exponential ratio (1,phi,phi^2)
ratio = np.exp(1)
f = {"f_IA":1, "f_IB":ratio, "f_T":1};        readout_1,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":ratio, "f_T":ratio};    readout_2,ex=three_node_network(f)
f = {"f_IA":1, "f_IB":ratio, "f_T":1+ratio};  readout_3,ex=three_node_network(f)
plt.subplot(121); plt.boxplot([readout_1["ABT"],readout_2["ABT"],readout_3["ABT"]])
plt.ylim([-0.001,0.02]); plt.xticks([1,2,3],[r"$(1,e) \rightarrow 1$", r"$(1,e) \rightarrow e$", r"$(1,e) \rightarrow 1+e$"]); plt.ylabel("Readout at target");
fig.savefig("Figure-3-Supp-Material.pdf", bbox_inches='tight')

ks13_exp = ks_2samp(readout_1["ABT"], readout_3["ABT"])
ks23_exp = ks_2samp(readout_2["ABT"], readout_3["ABT"])
print("Exp: Target=1 vs Target=e^2", ks13_exp)
print("Exp: Target=e vs Target=e^2", ks23_exp)
